In [1]:
from textProcessing import *
from evaluation import *
from sklearn.metrics.pairwise import cosine_similarity
from operator import itemgetter
import numpy as np
import main
import pandas as pd
import pickle
from num2words import num2words
from paths import *

In [3]:
tfidf_matrix = pickle.load(open(paths.dataset_one_tfidf_matrix, "rb"))
vectorizer = pickle.load(open(paths.dataset_one_model, "rb"))
qr_path = paths.dataset_one_question_forum
processed_queies = textProcessing.ds_pre_processing1(qr_path)
cleanedQueries1=[]
for doc in processed_queies:
    doc=' '.join(doc)
    cleanedQueries1.append(doc)

In [4]:
mappings = evaluation.golden_standard(paths.dataset_one_qas_forum)

In [5]:
print(mappings)

{'0': [116], '1': [32, 498, 1142, 3765, 4664, 6030, 7325, 7395, 9573], '2': [7204, 7205, 7206, 7207, 7210, 7212, 7213, 7214, 7215, 7217, 7225, 7226, 7228, 7229, 7231, 7232, 7236, 7239], '3': [427], '4': [6106, 6107, 6116, 6119, 6121, 6135], '5': [6, 5221, 6320, 9273], '6': [470, 471, 481, 2218], '7': [6216, 6217, 6219, 6220, 6227, 6228, 6231, 6236, 6238, 6246], '8': [2, 28, 6623, 7532], '9': [7486, 7487, 7488, 7493, 7494, 7500, 7512, 7514], '10': [9425, 9430, 9431, 9432], '11': [10003, 10005, 10007, 10009, 10013, 10015], '12': [1099, 1104, 1134, 1600], '13': [349, 350, 355, 366, 438, 511], '14': [287], '15': [7435, 7440, 7442, 7449, 7452, 7482], '16': [31, 3023], '17': [6449, 6451, 6452, 6453, 6457, 6459, 6461], '18': [6945, 6946, 6948, 6949, 6951, 6953, 6954, 6955], '19': [6707, 6708, 6709], '20': [11, 613, 1408], '21': [7719, 7721, 7722, 7724, 7726, 7727, 7729, 7735, 7737], '22': [485, 487, 491, 1929, 2327], '23': [1318, 1319, 1322, 1324, 1326, 1328, 1335], '24': [8287, 8288, 8289, 8

In [6]:
precision_all = 0.0
recall_all = 0.0
mean_avg_precision = 0.0
mean_reciprocal_rank = 0.0
rr = 0.0 
for query_id in mappings.keys():
    gold_standard = mappings.get(str(query_id))
    #query = qry_vectors[int(query_id)-1]
    results = {}
    model_output = []
    doc_id = 1
    query_vector = vectorizer.transform([cleanedQueries1[int(query_id)]])
    cos=cosine_similarity(tfidf_matrix,query_vector).flatten()
    map=dict()
    for x in range(cos.size):
        map[cos[x]]=x

    myKey = list(map.keys())
    myKey.sort(reverse=True)
    if len(myKey) >= 10:
        for i in range(10):
            model_output.append(map.get(myKey[i]))
    else:
        # Handle the case where myKey has fewer than 10 elements
        for key in myKey:
            model_output.append(map.get(key))
        
    
    avg_precision = evaluation.calculate_average_precision(model_output,gold_standard)
    print("avg_precision(" + str(query_id) + "): " + str(avg_precision))
    precision_all += avg_precision
    
    recall = evaluation.calculate_recall_at_k(model_output, gold_standard)
    print("recall(" + str(query_id) + "): " + str(recall))
    
    recall_all += recall
    
    reciprocal_rank = evaluation.calculate_reciprocal_rank(model_output, gold_standard)
    print("reciprocal_rank(" + str(query_id) + "): " + str(reciprocal_rank))
    print("-------------------------------")
    
    rr += reciprocal_rank
avg_recall = recall_all / float(len(mappings.keys()))
mean_avg_precision = precision_all / float(len(mappings.keys()))
mean_reciprocal_rank = rr / float(len(mappings.keys()))

print("-------------------------------")

print("mean average precision = " + str(mean_avg_precision))
print("mean reciprocal rank = " + str(mean_reciprocal_rank))
print("average recall = " + str(avg_recall))

avg_precision(0): 0
recall(0): 0.0
reciprocal_rank(0): 0
-------------------------------
avg_precision(1): 0.16666666666666666
recall(1): 0.1111111111111111
reciprocal_rank(1): 0.16666666666666666
-------------------------------
avg_precision(2): 0.5
recall(2): 0.05555555555555555
reciprocal_rank(2): 0.5
-------------------------------
avg_precision(3): 0
recall(3): 0.0
reciprocal_rank(3): 0
-------------------------------
avg_precision(4): 0
recall(4): 0.0
reciprocal_rank(4): 0
-------------------------------
avg_precision(5): 0.41666666666666663
recall(5): 0.5
reciprocal_rank(5): 0.3333333333333333
-------------------------------
avg_precision(6): 0.6666666666666666
recall(6): 0.75
reciprocal_rank(6): 1.0
-------------------------------
avg_precision(7): 0
recall(7): 0.0
reciprocal_rank(7): 0
-------------------------------
avg_precision(8): 1.0
recall(8): 0.5
reciprocal_rank(8): 1.0
-------------------------------
avg_precision(9): 0.125
recall(9): 0.125
reciprocal_rank(9): 0.125
--

_____________________
AFTER CLUSTERING 
________________________________________

In [14]:
kmeans = pickle.load(open(paths.clustered_dataset_one, "rb"))
vectorizer = pickle.load(open(paths.dataset_one_model, "rb"))
tfidf_matrix =  pickle.load(open(paths.dataset_one_tfidf_matrix, "rb"))
qr_path = paths.dataset_one_question_forum

In [15]:
processed_queies = textProcessing.ds_pre_processing1(qr_path)
cleanedQueries1=[]
for doc in processed_queies:
    doc=' '.join(doc)
    cleanedQueries1.append(doc)

In [16]:
mappings = evaluation.golden_standard(paths.dataset_one_qas_forum)

In [17]:
df_matrix = pd.DataFrame(tfidf_matrix)

cluster_map = pd.DataFrame()
cluster_map['doc_id'] = df_matrix.index.values
cluster_map['doc_vectors'] = df_matrix
cluster_map['cluster'] = kmeans.labels_

def get_vectors_of_one_cluster(query_cluster):
        
        one_cluster = cluster_map[cluster_map.cluster == query_cluster]
        
        return one_cluster

In [18]:
def cos_simi(cluster_vectors,query_vector,one_cluster_with_id):
        cos=cosine_similarity(cluster_vectors,query_vector).flatten()

        model_output = []
        map=dict()
        for x in range(cos.size):
            y=one_cluster_with_id['doc_id'].iloc[x]
            map[cos[x]]=y

        myKey = list(map.keys())
        myKey.sort(reverse=True)
        if len(myKey) >= 10:
            for i in range(10):
                model_output.append(map.get(myKey[i]))
        else:
            # Handle the case where myKey has fewer than 10 elements
            for key in myKey:
                model_output.append(map.get(key))
            
        return model_output


In [19]:
precision_all = 0.0
recall_all = 0.0
mean_avg_precision = 0.0
mean_reciprocal_rank = 0.0
rr = 0.0 

for query_id in mappings.keys():
    gold_standard = mappings.get(str(query_id))
    
    query_vector = vectorizer.transform([cleanedQueries1[int(query_id)]])
    #get query cluster
    query_cluster = kmeans.predict(np.array([query_vector.toarray().flatten()]))
    #get cluster vectors
    cluster_labels = kmeans.labels_  
    cluster_indices = np.where(cluster_labels == query_cluster)[0]
    cluster_vectors = tfidf_matrix[cluster_indices]
    #get document vectors of the same cluster as query
    one_cluster_with_id = get_vectors_of_one_cluster(query_cluster[0])
    #cosine similarity
    model_output = cos_simi(cluster_vectors,query_vector,one_cluster_with_id)


    avg_precision =evaluation.calculate_average_precision(model_output,gold_standard)
    print("avg_precision(" + str(query_id) + "): " + str(avg_precision))
    precision_all += avg_precision
    
    recall = evaluation.calculate_recall_at_k(model_output, gold_standard)
    print("recall(" + str(query_id) + "): " + str(recall))
    recall_all+=recall
    reciprocal_rank = evaluation.calculate_reciprocal_rank(model_output, gold_standard)
    print("reciprocal_rank(" + str(query_id) + "): " + str(reciprocal_rank))
    
    rr += reciprocal_rank
avg_recall = recall_all / float(len(mappings.keys()))
mean_avg_precision = precision_all / float(len(mappings.keys()))
mean_reciprocal_rank = rr / float(len(mappings.keys()))

print("-------------------------------")
print("mean average precision = " + str(mean_avg_precision))
print("mean reciprocal rank = " + str(mean_reciprocal_rank))   
print("average recall = " + str(avg_recall))

avg_precision(0): 0
recall(0): 0.0
reciprocal_rank(0): 0
avg_precision(1): 0.25
recall(1): 0.1111111111111111
reciprocal_rank(1): 0.25
avg_precision(2): 0.41111111111111115
recall(2): 0.16666666666666666
reciprocal_rank(2): 0.5
avg_precision(3): 0
recall(3): 0.0
reciprocal_rank(3): 0
avg_precision(4): 0.1
recall(4): 0.16666666666666666
reciprocal_rank(4): 0.1
avg_precision(5): 0.41666666666666663
recall(5): 0.5
reciprocal_rank(5): 0.3333333333333333
avg_precision(6): 0.6666666666666666
recall(6): 0.75
reciprocal_rank(6): 1.0
avg_precision(7): 0
recall(7): 0.0
reciprocal_rank(7): 0
avg_precision(8): 1.0
recall(8): 0.5
reciprocal_rank(8): 1.0
avg_precision(9): 0
recall(9): 0.0
reciprocal_rank(9): 0
avg_precision(10): 0.8055555555555555
recall(10): 0.75
reciprocal_rank(10): 1.0
avg_precision(11): 0
recall(11): 0.0
reciprocal_rank(11): 0
avg_precision(12): 0.3666666666666667
recall(12): 0.5
reciprocal_rank(12): 0.3333333333333333
avg_precision(13): 0.7595238095238095
recall(13): 0.83333333

________________________
WORD EMBEDDING
_______________________

In [2]:
embedded_docs = paths.embedded_dataset_one
qr_path = paths.dataset_one_question_forum

In [3]:
docs = pickle.load(open(embedded_docs, "rb"))

In [4]:
mappings = evaluation.golden_standard(paths.dataset_one_qas_forum)

In [7]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [5]:
processed_queies = textProcessing.ds_pre_processing1(qr_path)
cleanedQueries1=[]
for doc in processed_queies:
    doc=' '.join(doc)
    cleanedQueries1.append(doc)

In [ ]:
precision_all = 0.0
recall_all = 0.0
mean_avg_precision = 0.0
mean_reciprocal_rank = 0.0
rr = 0.0 
for query_id in mappings.keys():
    gold_standard = mappings.get(str(query_id))
    #query = qry_vectors[int(query_id)-1]
    results = {}
    model_output = []
    doc_id = 1
    #query_vector = vectorizer.transform([cleanedQueries1[int(query_id)]])
    query_vector = [nlp(str([cleanedQueries1[int(query_id)]])).vector]
    cos=cosine_similarity(docs,query_vector).flatten()
    map=dict()
    for x in range(cos.size):
        map[cos[x]]=x

    myKey = list(map.keys())
    myKey.sort(reverse=True)
    if len(myKey) >= 10:
        for i in range(10):
            model_output.append(map.get(myKey[i]))
    else:
        # Handle the case where myKey has fewer than 10 elements
        for key in myKey:
            model_output.append(map.get(key))
        
    
    avg_precision = evaluation.calculate_average_precision(model_output,gold_standard)
    print("avg_precision(" + str(query_id) + "): " + str(avg_precision))
    precision_all += avg_precision
    
    recall = evaluation.calculate_recall_at_k(model_output, gold_standard)
    print("recall(" + str(query_id) + "): " + str(recall))
    
    reciprocal_rank = evaluation.calculate_reciprocal_rank(model_output, gold_standard)
    print("reciprocal_rank(" + str(query_id) + "): " + str(reciprocal_rank))
    
    rr += reciprocal_rank

mean_avg_precision = precision_all / float(len(mappings.keys()))
mean_reciprocal_rank = rr / float(len(mappings.keys()))

print("-------------------------------")
print("mean average precision = " + str(mean_avg_precision))
print("mean reciprocal rank = " + str(mean_reciprocal_rank))   
